In [8]:
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup
import json
import re
import numpy as np

In [9]:
steamspy_url = 'https://steamspy.com/api.php?request=all'
r = requests.get(steamspy_url)

In [3]:
print(r.json())

{}


In [10]:
client = MongoClient('localhost', 27017)
db = client.steamspy
test = db.test

In [20]:
print(client)
print(db)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'steamspy')


In [11]:
for item in r.json().values():
    test.insert(item)

In [12]:
data = pd.DataFrame(list(test.find()))

In [27]:
print(r.json())

{}


In [13]:
#replacing owners column with lower limit owners and upper limit owners column
for idx in range(data.shape[0]):
    owners = data.iloc[idx]['owners'].split('..')
    for i in range(len(owners)):
        owners[i] = owners[i].replace(',','')
        owners[i] = owners[i].replace(' ','')
        owners[i] = int(owners[i])
    data.at[idx, 'llOwners'] = owners[0]
    data.at[idx, 'ulOwners'] = owners[1]

In [14]:
#Params: genre(string) represents the genre you want to get data of
#        collection(database collection) represents database to store the genre data in
#This function gets the data of a certain genre from the steamspy website and puts it in the given collection
def storeGenre(genre, collection):
    steamspy_url = 'https://steamspy.com/api.php?request=genre&genre={}'.format(genre)
    r = requests.post(steamspy_url)
    for item in r.json().values():
        collection.insert(item)

In [15]:
#Params: tag(string) represents the tag you want to get data of
#        collection(database collection) represents database to store the gtag data in
#This function gets the data of a certain tag from the steamspy website and puts it in the given collection
def storeTag(collection, tag):
    steamspy_url = 'https://steamspy.com/api.php?request=tag&tag={}'.format(tag)
    r = requests.post(steamspy_url)
    for item in r.json().values():
        collection.insert(item)

In [16]:
action = db.action

In [17]:
storeGenre('Action', action)

In [18]:
earlyaccess = db.earlyaccess
storeGenre('Early+Access', earlyaccess)

In [19]:
#creating databases for all the genres
strategy = db.strategy
rpg = db.rpg
indieG = db.indieG
adventure = db.adventure
sports = db.sports
simulation = db.simulation
exEarlyAccess = db.exEarlyAccess
mmo = db.mmo
free = db.free

In [20]:
#storing all genre data in databases
storeGenre('Strategy', strategy)
storeGenre('RPG', rpg)
storeGenre('Indie', indieG)
storeGenre('Adventure', adventure)
storeGenre('Sports', sports)
storeGenre('Simulation', simulation)
storeGenre('Ex+Early+Access',exEarlyAccess)
storeGenre('Free', free)

In [21]:
storeGenre('Massively', mmo)

In [22]:
data['Genres'] = np.empty((len(data), 0)).tolist()
data['Tags'] = np.empty((len(data), 0)).tolist()

In [23]:
#Params: df(dataframe) represents the data to append to
#        genreDB(database) represents the genre database that you are adding
#        genre(string) the genre you are appending
#This function appends to the Genre column of df if the game is in the certain genre
def appendGenre(df, genreDB, genre):
    genreData = pd.DataFrame(list(genreDB.find()))
    genreData = genreData.drop(columns = ['score_rank', 'owners', '_id'])
    df['{}'.format(genre)] = data.appid.isin(genreData.appid).astype(int)
    for idx in range(df.shape[0]):
        if df.iloc[idx]['{}'.format(genre)] == 1:
            df.at[idx, 'Genres'].append('{}'.format(genre))
    df.drop(columns = '{}'.format(genre), axis=1, inplace = True)
    
#     df = pd.merge(df, genreData, on=['appid','average_2weeks','average_forever','developer',
#                                    'discount', 'initialprice','median_2weeks','median_forever',
#                                    'name', 'negative', 'positive', 'price', 'publisher',
#                                    'userscore'], how='left', indicator='{}'.format(genre))

In [24]:
#Params: df(dataframe) represents the data to append to
#        tagDB(database) represents the tag database that you are adding
#        tag(string) the tag you are appending
#This function appends to the Tags column of df if the game includes a certain tag
def appendTag(df, tagDB, tag):
    tagData = pd.DataFrame(list(tagDB.find()))
    tagData = tagData.drop(columns = ['score_rank', 'owners', '_id'])
    df['{}'.format(tag)] = data.appid.isin(tagData.appid).astype(int)
    for idx in range(df.shape[0]):
        if df.iloc[idx]['{}'.format(tag)] == 1:
            df.at[idx, 'Tags'].append('{}'.format(tag))
    df.drop(columns = '{}'.format(tag), axis=1, inplace = True)

In [214]:
dataMMO = pd.DataFrame(list(mmo.find()))
dataMMO.head()

""


In [25]:
#appending the genres
appendGenre(data, action, 'Action')
appendGenre(data, earlyaccess, 'Early Access')
appendGenre(data, strategy, 'Strategy')
appendGenre(data, rpg, 'RPG')
appendGenre(data, indieG, 'Indie')
appendGenre(data, adventure, 'Adventure')
appendGenre(data, sports, 'Sports')
appendGenre(data, simulation, 'Simulation')
appendGenre(data, exEarlyAccess, 'Ex Early Access')

In [26]:
appendGenre(data, mmo, 'MMO')
appendGenre(data, free, 'Free')

In [27]:
#screating databases for the tags
singleplayerT = db.singleplayerT
casualT = db.casualT
multiplayerT = db.multiplayerT
soundtrackT = db.soundtrackT #Great Soundtrack
atmosphericT = db.atmosphericT
twodT = db.twodT #2D
puzzleT = db.puzzleT
openWorldT = db.openWorldT
storyRichT = db.storyRichT
coopT = db.coopT #co-op
difficultT = db.difficultT
shooterT = db.shooterT
scifiT = db.scifiT #sci-fi
firstPersonT = db.firstPersonT #First-Person
horrorT = db.horrorT
vrT = db.vrT
animeT = db.animeT
pixelGraphicsT = db.pixelGraphicsT
funnyT = db.funnyT
fantasyT = db.fantasyT
platformerT = db.platformerT
femaleProtagonistT = db.femaleProtagonistT
fpsT = db.fpsT
survivalT = db.survivalT
goreT = db.goreT
violentT = db.violentT
sandboxT = db.sanboxT
comedyT = db.comedyT
arcadeT = db.arcadeT
classicT = db.classicT
nudityT = db.nudityT
onlineCoopT = db.onlineCoopT #Online Co-Op
tacticalT = db.tacticalT
zombiesT = db.zombiesT
teamBasedT = db.teamBasedT #Team-Based
pveT = db.pveT #PvE
pvpT = db.pvpT #PvP
warT = db.warT
controllerT = db.controllerT

In [28]:
storeTag(singleplayerT, 'Singleplayer')
storeTag(casualT, 'Casual')
storeTag(multiplayerT, 'Multiplayer')
storeTag(soundtrackT, 'Great+Soundtrack')
storeTag(atmosphericT, 'Atmospheric')
storeTag(twodT, '2D')
storeTag(puzzleT, 'Puzzle')
storeTag(openWorldT, 'Open+World')
storeTag(storyRichT, 'Story+Rich')
storeTag(coopT, 'Co-op')
storeTag(difficultT, 'Difficult')
storeTag(shooterT, 'Shooter')
storeTag(scifiT, 'Sci-fi')
storeTag(firstPersonT, 'First-Person')
storeTag(horrorT, 'Horror')
storeTag(vrT, 'VR')
storeTag(animeT, 'Anime')
storeTag(pixelGraphicsT, 'Pixel+Graphics')
storeTag(funnyT, 'Funny')
storeTag(fantasyT, 'Fantasy')
storeTag(platformerT, 'Platformer')
storeTag(femaleProtagonistT, 'Female+Protagonist')
storeTag(fpsT, 'FPS')
storeTag(survivalT, 'Survival')
storeTag(goreT, 'Gore')
storeTag(violentT, 'Violent')
storeTag(sandboxT, 'Sandbox')
storeTag(comedyT, 'Comedy')
storeTag(arcadeT, 'Arcade')
storeTag(classicT, 'Classic')
storeTag(nudityT, 'Nudity')
storeTag(onlineCoopT, 'Online+Co-op')
storeTag(tacticalT, 'Tactical')
storeTag(zombiesT, 'Zombies')
storeTag(teamBasedT, 'Team-Based')
storeTag(pveT, 'PvE')
storeTag(pvpT, 'PvP')
storeTag(warT, 'War')
storeTag(controllerT, 'Controller')

In [29]:
appendTag(data, singleplayerT, 'Singleplayer')
appendTag(data, casualT, 'Casual')
appendTag(data, multiplayerT, 'Multiplayer')
appendTag(data, soundtrackT, 'Great+Soundtrack')
appendTag(data, atmosphericT, 'Atmospheric')
appendTag(data, twodT, '2D')
appendTag(data, puzzleT, 'Puzzle')
appendTag(data, openWorldT, 'Open+World')
appendTag(data, storyRichT, 'Story+Rich')
appendTag(data, coopT, 'Co-op')
appendTag(data, difficultT, 'Difficult')
appendTag(data, shooterT, 'Shooter')
appendTag(data, scifiT, 'Sci-fi')
appendTag(data, firstPersonT, 'First-Person')
appendTag(data, horrorT, 'Horror')
appendTag(data, vrT, 'VR')
appendTag(data, animeT, 'Anime')
appendTag(data, pixelGraphicsT, 'Pixel+Graphics')
appendTag(data, funnyT, 'Funny')
appendTag(data, fantasyT, 'Fantasy')
appendTag(data, platformerT, 'Platformer')
appendTag(data, femaleProtagonistT, 'Female+Protagonist')
appendTag(data, fpsT, 'FPS')
appendTag(data, survivalT, 'Survival')
appendTag(data, goreT, 'Gore')
appendTag(data, violentT, 'Violent')
appendTag(data, sandboxT, 'Sandbox')
appendTag(data, comedyT, 'Comedy')
appendTag(data, arcadeT, 'Arcade')
appendTag(data, classicT, 'Classic')
appendTag(data, nudityT, 'Nudity')
appendTag(data, onlineCoopT, 'Online+Co-op')
appendTag(data, tacticalT, 'Tactical')
appendTag(data, zombiesT, 'Zombies')
appendTag(data, teamBasedT, 'Team-Based')
appendTag(data, pveT, 'PvE')
appendTag(data, pvpT, 'PvP')
appendTag(data, warT, 'War')
appendTag(data, controllerT, 'Controller')

In [35]:
data = data.drop(columns = ['_id', 'discount','owners'])

In [ ]:
clean = db.clean

In [ ]:
records = json.loads(data.T.to_json()).values() #store clean dataframe in mongo
clean.insert(records)

In [45]:
gamefeatures= pd.read_csv("gamefeatures.csv")